In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from dataclasses import dataclass
import numpy as np
import json

In [2]:

class LayerNorm(nn.Module):
    def __init__(self, ndim, bias):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(ndim))
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None
    def forward(self, x):
        return F.layer_norm(x, self.weight.shape, self.weight, self.bias, 1e-5)

class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=config.bias)
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
        self.attn_dropout = nn.Dropout(config.dropout)
        self.resid_dropout = nn.Dropout(config.dropout)
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.flash = hasattr(F, 'scaled_dot_product_attention')
        if not self.flash:
            self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                       .view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size()
        q, k, v = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)

        if self.flash:
            y = F.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.attn_dropout.p if self.training else 0.0, is_causal=True)
        else:
            att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
            att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))
            att = F.softmax(att, dim=-1)
            att = self.attn_dropout(att)
            y = att @ v

        y = y.transpose(1, 2).contiguous().view(B, T, C)
        y = self.resid_dropout(self.c_proj(y))
        return y

class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd, bias=config.bias)
        self.gelu = nn.GELU()
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias)
        self.dropout = nn.Dropout(config.dropout)
    def forward(self, x):
        return self.dropout(self.c_proj(self.gelu(self.c_fc(x))))

class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln1 = LayerNorm(config.n_embd, config.bias)
        self.attn = CausalSelfAttention(config)
        self.ln2 = LayerNorm(config.n_embd, config.bias)
        self.mlp = MLP(config)
    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x

@dataclass
class GPTConfig:
    block_size: int
    vocab_size: int
    n_layer: int
    n_head: int
    n_embd: int
    dropout: float = 0.0
    bias: bool = True

class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.transformer = nn.ModuleDict(dict(
            wte=nn.Embedding(config.vocab_size, config.n_embd),
            wpe=nn.Embedding(config.block_size, config.n_embd),
            drop=nn.Dropout(config.dropout),
            h=nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f=LayerNorm(config.n_embd, config.bias),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        self.transformer.wte.weight = self.lm_head.weight  # weight tying

        self.apply(self._init_weights)
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                nn.init.normal_(p, mean=0.0, std=0.02 / math.sqrt(2 * config.n_layer))

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.config.block_size
        pos = torch.arange(0, t, dtype=torch.long, device=device)

        tok_emb = self.transformer.wte(idx)
        pos_emb = self.transformer.wpe(pos)
        x = self.transformer.drop(tok_emb + pos_emb)
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        if targets is not None:
            logits = self.lm_head(x)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
            return logits, loss
        else:
            logits = self.lm_head(x[:, [-1], :])
            return logits, None

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        """
        Generate tokens given a conditioning sequence.
        idx: Tensor of shape (B, T)
        """
        for _ in range(max_new_tokens):
            idx_cond = idx if idx.size(1) <= self.config.block_size else idx[:, -self.config.block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / temperature
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx


In [3]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


True
NVIDIA GeForce RTX 4060 Laptop GPU


In [3]:
numb = 50257
# 50257
config = GPTConfig(
    vocab_size=numb,     # use the tokenizer's vocab size
    block_size=128,       # or whatever context size you're training with
    n_layer=6,
    n_head=6,
    n_embd=384,
    dropout=0.1,
    bias=True
)

model = GPT(config)

In [1]:
import os
import torch

# Встановіть debugging режим
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Перевірте CUDA
print(f"CUDA доступна: {torch.cuda.is_available()}")
print(f"CUDA версія: {torch.version.cuda}")

try:
    # Створіть тестовий тензор
    test_tensor = torch.tensor([1.0]).cuda()
    print("CUDA працює нормально")
    del test_tensor
except Exception as e:
    print(f"CUDA проблема: {e}")

CUDA доступна: True
CUDA версія: 12.1
CUDA працює нормально


In [4]:
def safe_model_load(model, path, numb):
    try:
        # Завантажте на CPU спочатку
        print("Завантажую на CPU...")
        checkpoint = torch.load(f"{path}/gpt_1.3_new_gpt_50ep.pt", map_location='cpu')
        # checkpoint = torch.load(f"{path}/TinyStories_{numb}.pt", map_location='cpu')
        # checkpoint = torch.load(f"{path}/114mb_20000.pt", map_location='cpu')
        
        # Завантажте state dict
        model.load_state_dict(checkpoint)
        print("State dict завантажено")
        
        # Очистіть checkpoint з пам'яті
        del checkpoint
        torch.cuda.empty_cache()
        
        # Поступово перенесіть на GPU
        print("Переношу на GPU...")
        model = model.cuda()
        model.eval()
        
        print("Модель успішно завантажена на GPU")
        return model
        
    except Exception as e:
        print(f"Помилка завантаження: {e}")
        print("Залишаю модель на CPU")
        model.eval()
        return model


In [5]:
model = safe_model_load(model, "../models", numb)
device = next(model.parameters()).device
print(f"Модель на пристрої: {device}")

Завантажую на CPU...
State dict завантажено
Переношу на GPU...


C:\Users\matsk\AppData\Local\Temp\ipykernel_17036\2764060940.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(f"{path}/gpt_1.3_new_gpt_50ep.pt", m

Модель успішно завантажена на GPU
Модель на пристрої: cuda:0


In [6]:
import tiktoken
enc = tiktoken.get_encoding("gpt2")

block_size = config.block_size  # =128

In [7]:
SEED = 12345
import random as _random
_random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [8]:
def generate_response(instruction, max_new_tokens=100, temperature=0.6, top_k=40):
    prompt = f"question: {instruction}\nanswer:"
    input_ids = enc.encode_ordinary(prompt)
    input_ids = input_ids[:config.block_size]
    input_tensor = torch.tensor(input_ids, dtype=torch.long)[None].to("cuda")

    model.eval()
    with torch.no_grad():
        for _ in range(max_new_tokens):
            if input_tensor.shape[1] > config.block_size:
                input_tensor = input_tensor[:, -config.block_size:]

            logits, _ = model(input_tensor)
            logits = logits[:, -1, :] / temperature
            if top_k is not None:
                values, _ = torch.topk(logits, top_k)
                logits[logits < values[:, [-1]]] = -float("inf")

            probs = torch.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            input_tensor = torch.cat([input_tensor, next_token], dim=1)

            # Зупиняємо генерацію, якщо згенеровано токен <|END|> (50256)
            if next_token.item() == 50256:
                break

    output_tokens = input_tensor[0].tolist()
    generated = enc.decode(output_tokens[len(input_ids):])
    return generated.strip().replace("<|END|><|endoftext|>", "")  # Видаляємо <|END|> із виводу


In [9]:
with open("../eval_selection/hella_dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)
input_pairs = [{"instruction": item["instruction"], "response": item["response"]} for item in data]
print(input_pairs[0]["instruction"])

You turned on PowerCool in the refrigerator. What happens next? A) The refrigerator will always stay at maximum cooling. B) After a few hours, it returns to the previous temperature. C) The refrigerator stops working completely.


In [26]:
for pair in input_pairs:
    instruction = pair["instruction"]
    generated = generate_response(instruction)

    print("Instruction:", instruction)
    print("Generated:", generated)
    print("Expected:", pair["response"])
    print("-" * 50)

Instruction: You turned on PowerCool in the refrigerator. What happens next? A) The refrigerator will always stay at maximum cooling. B) After a few hours, it returns to the previous temperature. C) The refrigerator stops working completely.
Generated: For specific instructions on specialized compartments, it is recommended to keep the freezer’s ideal for quickly freezing bulk items, ensuring airtight containers to keep the temperature stable.
Expected: B
--------------------------------------------------
Instruction: A person left the fridge door open for a long time. What will likely happen? A) The temperature inside will rise and food may spoil. B) The refrigerator will automatically close the door. C) The temperature will drop further.
Generated: To ensure optimal food safety and potential contamination, ensure the door isn’t left open for extended periods to keep the temperature stable; routinely clean surfaces that come into contact with food; wash water tanks if unused for 48 ho

In [19]:
with open("../eval_selection/piqa_dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)
input_pairs = [{"instruction": item["instruction"], "response": item["response"]} for item in data]
print(input_pairs[0]["instruction"])

How should eggs be stored in the fridge – in the door or in a carton on a shelf?


In [20]:
for pair in input_pairs:
    instruction = pair["instruction"]
    generated = generate_response(instruction)

    print("Instruction:", instruction)
    print("Generated:", generated)
    print("Expected:", pair["response"])
    print("-" * 50)

Instruction: How should eggs be stored in the fridge – in the door or in a carton on a shelf?
Generated: Leaving the door open for extended times can raise the temperature considerably, potentially compromising food safety and reducing the appliance's efficiency.
Expected: in a carton on a shelf
--------------------------------------------------
Instruction: What is the safe way to clean the fridge – unplug it first or wash inside with a water jet?
Generated: I apologize, but I am a refrigerator assistant and cannot help with water rules.
Expected: unplug it first
--------------------------------------------------
Instruction: How can frost be avoided – by opening the door often or by ensuring proper ventilation and temperature?
Generated: When installing, make sure not to place the refrigerator door upside down, since this can result in a multi-socket adapter or operational problems, making it important to keep the appliance stable and undamaged.
Expected: by ensuring proper ventilati

In [15]:
with open("../eval_selection/boolq_dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)
input_pairs = [{"instruction": item["instruction"], "response": item["response"]} for item in data]
print(input_pairs[0]["instruction"])

Can the refrigerator be transported lying down?


In [16]:
for pair in input_pairs:
    instruction = pair["instruction"]
    generated = generate_response(instruction)

    print("Instruction:", instruction)
    print("Generated:", generated)
    print("Expected:", pair["response"])
    print("-" * 50)

Instruction: Can the refrigerator be transported lying down?
Generated: I apologize, but I am a refrigerator assistant and cannot help with athletic guidance.
Expected: no
--------------------------------------------------
Instruction: Should the refrigerator only be connected to a grounded outlet?
Generated: The refrigerator should only be used for storing food, and storing volatile or flammable chemicals inside is prohibited. Doing so could trigger fires or explosions, making it important to follow this safety rule.
Expected: yes
--------------------------------------------------
Instruction: Can raw meat be stored on a shelf without a container?
Generated: Store raw meat and fish in suitable containers helps prevent them from touching other foods or leaking fluids, which is essential for preventing cross-contamination and ensuring food safety.
Expected: no
--------------------------------------------------
Instruction: Does PowerCool turn off automatically after a few hours?
Generat

In [17]:
with open("../eval_selection/WinoGrande_dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)
input_pairs = [{"instruction": item["instruction"], "response": item["response"]} for item in data]
print(input_pairs[0]["instruction"])

The user placed raw meat in a container and put it in the fridge so _ would not contaminate other food. meat container


In [18]:
for pair in input_pairs:
    instruction = pair["instruction"]
    generated = generate_response(instruction)

    print("Instruction:", instruction)
    print("Generated:", generated)
    print("Expected:", pair["response"])
    print("-" * 50)

Instruction: The user placed raw meat in a container and put it in the fridge so _ would not contaminate other food. meat container
Generated: Store raw meat and fish in suitable containers to prevent their juices from dripping onto other foods and to maintain safe, contamination-free storage.
Expected: meat
--------------------------------------------------
Instruction: The child opened the fridge door, although _ was heavy. child door
Generated: Do not hang on the refrigerator door, as this may lead to benzene or operational problems, ensuring the door doesn't pinch the cable.
Expected: door
--------------------------------------------------
Instruction: The technician checked the compressor because _ was noisy. technician compressor
Generated: I apologize, but I am a refrigerator assistant and cannot help with historical facts.
Expected: compressor
--------------------------------------------------
Instruction: The user put a bottle in the fridge door because _ was tall. bottle door